# mRAG-01

## Imports

In [1]:
import asyncio
import requests
import httpx

## Variables

In [71]:
# Define Ollama endpoints
ollama_base_url = "http://127.0.0.1:11434"
ollama_chat = "/api/chat"
ollama_embedding = "/api/embeddings"

# Define Ollama models
embedding_model = "mxbai-embed-large"
llm_model = "llama3.1:8b"

## Key Functions

### LLM

In [ ]:
async def get_llm_response(in_text, timeout=60.0):
    data = {
        "model": llm_model,
        "messages": [
            {"role": "user", "content": in_text}
        ],
        "stream": False
    }
    
    try:
        # Create a client with an increased timeout
        async with httpx.AsyncClient(timeout=httpx.Timeout(timeout)) as client:
            response = await client.post((ollama_base_url + ollama_chat), json=data)
            response.raise_for_status()
            
            result = response.json()
            
            # Extract the response text based on Ollama's API structure
            if "message" in result and "content" in result["message"]:
                return result["message"]["content"].strip()
            else:
                print('Unexpected response structure:', result)
                return None
                
    except httpx.ReadTimeout:
        print("Request timed out. The Ollama server might be busy or the model is taking too long to respond.")
        return None
    except httpx.ConnectError:
        print("Could not connect to the Ollama server. Make sure it's running at http://127.0.0.1:11434.")
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [72]:
async def get_embeddings(text, model=, timeout=60.0):
    """
    Get embeddings for the provided text using Ollama's API.
    
    Args:
        text: The text to generate embeddings for
        model: The model to use for embeddings (default: "llama3.1:8b")
        timeout: Timeout in seconds (default: 60 seconds)
    
    Returns:
        A list of embedding values or None if an error occurs
    """
    data = {
        "model": model,
        "prompt": text,
        "options": {
            "temperature": 0.0  # Lower temperature for more deterministic embeddings
        }
    }
    
    try:
        # Create a client with an increased timeout
        async with httpx.AsyncClient(timeout=httpx.Timeout(timeout)) as client:
            response = await client.post((ollama_base_url + ollama_embedding), json=data)
            response.raise_for_status()
            
            result = response.json()
            
            # Extract the embeddings from the response
            if "embedding" in result:
                return result["embedding"]
            else:
                print('No embeddings found in response:', result)
                return None
                
    except httpx.ReadTimeout:
        print("Request timed out. The Ollama server might be busy or the model is taking too long to respond.")
        return None
    except httpx.ConnectError:
        print("Could not connect to the Ollama server. Make sure it's running at http://127.0.0.1:11434.")
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

### Vector Database

In [77]:
async def load_matrix():
    return null

In [78]:
async def append_matrix():
    return null

In [79]:
async def find_nearest_k(k):
    return null

## Testing

In [60]:
prompt = "This is another test prompt"

In [61]:
response = await get_llm_response(prompt)

In [62]:
print(response)

It looks like you're testing the system. I'm ready when you are. What would you like to talk about or do?


In [73]:
embeddings = await get_embeddings(prompt)

In [74]:
print(embeddings

[0.5756283402442932, 0.025145139545202255, 0.017586376518011093, 0.2662537097930908, -0.9467823505401611, -0.378476619720459, -0.19218437373638153, -0.05940169841051102, 0.22314892709255219, 1.241896629333496, -0.1529102921485901, 0.5244695544242859, -0.2150721251964569, -0.4756650924682617, -0.4049847424030304, -0.7585371732711792, -0.22175472974777222, -0.043068669736385345, -0.5012223720550537, -0.9111211895942688, -0.026720233261585236, 0.5962637662887573, -0.7997405529022217, -0.26611700654029846, -0.48238348960876465, -0.07176170498132706, -0.26466166973114014, 0.40670204162597656, 0.30144816637039185, 0.5829399824142456, -0.42821004986763, 0.5520590543746948, 0.02861153334379196, -0.6617875099182129, -0.13064660131931305, -0.6627650856971741, 1.2395473718643188, -0.12948891520500183, -0.02363390475511551, -0.4530979096889496, -0.49942171573638916, 0.33722245693206787, -0.047672830522060394, -0.788583517074585, -1.2296503782272339, -0.8267309665679932, -0.5478063225746155, -0.203